# Creating ChromaDB


In [19]:
import dotenv
import chromadb.utils.embedding_functions as embedding_functions
from google import genai
import os
import chromadb
import pandas as pd

In [20]:
client = chromadb.PersistentClient(path="./chroma_db")
collection = client.get_or_create_collection(name="cryptonews_collection")

In [21]:
df = pd.read_csv("../creating_database/cryptonews-cleaned.csv")

In [5]:
dotenv.load_dotenv()

openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=os.getenv("OPENAI_API_KEY"),
    model_name='text-embedding-3-small'
)

In [22]:
articles = pd.read_csv("../creating_database/cryptonews-cleaned.csv")
articles.head()

,id,datetime,url,label,summary,cause,effect,sentiment,coins_mentioned
0,1,2022-10-14,https://cryptonews.com/news/mit-research-scien...,1,MIT Research Scientist Lex Fridman and other B...,Discussion sparked by the 2022 Nobel Prize in ...,"No specific price movements mentioned, but the...",bullish,BTC
1,2,2022-10-14,https://cryptonews.com/news/bitcoin-price-and-...,1,The cryptocurrency market experienced a signif...,US CPI inflation data came in higher than expe...,Total crypto market cap increased 2.79% to $94...,bullish,"BTC,ETH,ENS,USTC,LDO,CVX,TAMA,IMPT"
2,3,2022-10-15,https://cryptonews.com/news/bitcoin-price-ethe...,1,The cryptocurrency market experienced a bearis...,Fed rate hike sentiment strengthened due to be...,Global crypto market cap declined 1.9% to $919...,mixed,"BTC,ETH,HT,LUNA,TRX,LEO,USTC,SNX,IMPT"
3,4,2022-10-17,https://cryptonews.com/news/bitcoin-price-fore...,0,Bitcoin recovered slightly over 1% during the ...,Multiple factors including Bitcoin mining dela...,Bitcoin recovered 1% during US session from op...,mixed,"BTC,IMPT"
4,5,2022-10-17,https://cryptonews.com/news/south-korean-prose...,0,South Korean prosecutors announced plans to pu...,South Korean prosecutors implementing advanced...,"No specific price movements mentioned, but inc...",bearish,BTC


In [23]:
five_articles = articles['summary'][:5].tolist()
five_articles

["MIT Research Scientist Lex Fridman and other Bitcoin supporters argue that Satoshi Nakamoto, Bitcoin's creator, should receive the Nobel Memorial Prize in Economic Sciences. This argument emerged after the 2022 Nobel Prize in Economic Sciences was awarded to three American economists for their work on bank runs. The crypto community believes Nakamoto deserves recognition for creating a monetary system that provides economic freedom, though anonymous recipients cannot receive Nobel Prizes.",
 'The cryptocurrency market experienced a significant rally on October 14, 2022, with total market cap rising 2.79% to $944.24 billion despite stronger-than-expected US CPI inflation data. Bitcoin rebounded 3.79% from triple bottom support at $18,248 to trade at $19,807, while Ethereum gained 3.28% to trade above the psychological $1,300 level. Top performers included Ethereum Name Service (+17%), TerraClassicUSD (+12.42%), and Lido DAO (+12.84%). The market showed resilience by trading risk-on de

In [14]:
embeddings = openai_ef(five_articles)

In [24]:
embeddings

[array([ 0.00871456, -0.0436893 ,  0.01219179, ..., -0.02830851,
         0.05887385,  0.03417136], shape=(1536,), dtype=float32),
 array([ 0.02063979, -0.03249334,  0.03193156, ..., -0.00121134,
         0.01260634,  0.0158759 ], shape=(1536,), dtype=float32),
 array([-0.01771403, -0.01615268,  0.0386631 , ..., -0.01082275,
        -0.01106987,  0.03493383], shape=(1536,), dtype=float32),
 array([-0.00230053, -0.01412417,  0.04577401, ..., -0.01345963,
        -0.01231078,  0.0391512 ], shape=(1536,), dtype=float32),
 array([-0.01705444, -0.03082594,  0.02279961, ..., -0.03196049,
        -0.03027073, -0.00036586], shape=(1536,), dtype=float32)]

In [25]:
for index, row in df.iterrows():
    collection.add(
        ids=[str(row['id'])],
        documents=[row['summary']],
        embeddings=[index],
        metadatas=[{
            "id": row['id'],
            "datetime": row['datetime'],
            "url": row['url']
        }],
    )

In [ ]:
query_text = "South Korean government to regulate crypto exchanges"